In [0]:
import keras
from keras.models import load_model
#from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import os
import pickle
import numpy as np
import h5py
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import pandas as pd
from PIL import Image
from keras import metrics
import random

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
input_img = Input(shape=(2048, 2048, 3))
x = Conv2D(4, (7, 7), padding='same')(input_img)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (5, 5), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

#x = Conv2D(16, (3, 3), padding='same')(x)
#x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

#x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (16,16),padding="valid",strides=16)(x)
#x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(64)(x)
x = Dropout(0.5)(x)
x = Dense(3)(x)
out = Activation("softmax")(x)
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

weights_path = '/content/drive/My Drive/research_/weight/best_weight_0507.h5' 
assert os.path.exists(weights_path), 'Model weights not found '
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []

for k, name in enumerate(layer_names):
    if k >= 9:
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

#for layer in model.layers[:9]:
        #layer.trainable = False


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048, 2048, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2048, 2048, 4)     592       
_________________________________________________________________
activation_1 (Activation)    (None, 2048, 2048, 4)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1024, 1024, 4)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1024, 1024, 8)     808       
_________________________________________________________________
activation_2 (Activat

In [0]:
input_img = Input(shape=(2048, 2048, 3))
x = Conv2D(4, (7, 7), padding='same')(input_img)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (5, 5), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(x)
x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

x = MaxPooling2D((2, 2), padding='same')(x)

#x = Conv2D(16, (3, 3), padding='same')(x)
#x = Activation('relu')(x)
#x = BatchNormalization()(x)
#x = Dropout(0.25)(x)

#x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(512, (16,16),padding="valid",strides=16)(x)
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(128)(x)
x = Dropout(0.5)(x)
x = Dense(3)(x)
out = Activation("softmax")(x)
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2048, 2048, 3)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2048, 2048, 4)     592       
_________________________________________________________________
activation_1 (Activation)    (None, 2048, 2048, 4)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1024, 1024, 4)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1024, 1024, 8)     808       
_________________________________________________________________
activation_2 (Activat

In [0]:
modelCheckpoint = ModelCheckpoint(filepath ="/content/drive/My Drive/research_/weight/weight_0529_03.h5",
                                  monitor = "val_loss",
                                  verbose=1,
                                  save_best_only = True,
                                  save_weights_only = False,
                                  mode = "min",
                                  period =1)

In [0]:
class ImageDataGenerator(object):
  def __init__(self):
    self.reset()

  def reset(self):
    self.images = []
    self.labels = []
    self.labelsx = []
  #file_pathはメタCSVでーたのこと
  
  def flow_from_directory(self, file_path, classes, batch_size=4):
    blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
    blunc_l=np.empty((batch_size), np.float32)
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes = {v: i for i, v in enumerate(sorted(classes))}
    file_path = pd.read_csv(file_path)
    boss_dir = "/mnt/Doc"
    while True:
      # ディレクトリから画像のパスを取り出す
      count = 0
      count = 0
      bin8 = np.arange(batch_size,length,batch_size)
      hoge = np.arange(length)
      #np.random.shuffle(hoge)
      batch10 = np.split(hoge, bin8)
      #print(len(batch10))
      for ic, bbb in enumerate(batch10):
      for i in range(len(file_path)):
        # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
        path = file_path.loc[i]["path1"]
        print(i)
        with Image.open(path) as f:
          self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
          # ファイル名からラベルを取り出し、配列(self.labels)に格納
          #_, y = path.stem.split('_')
        label = file_path.loc[i]["type"]
        self.labels.append(to_categorical(classes[label],len(classes)))
        self.labelsx.append(to_categorical(classes[label],len(classes)))
          # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
          # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
        if len(self.images)== batch_size:
          #print(len(self.images))
          inputs = np.asarray(self.images,dtype=np.float32) 
          targets = np.asarray(self.labels,dtype=np.float32)
            #print(len(self.images))
          count+=1
            #if count%1000==0:
            #print("n*1000!")
          self.reset()
          yield inputs, targets
   

In [0]:
class ImageDataGenerator3(object):
  def __init__(self):
    self.reset()

  def reset(self):
    self.images = []
    self.labels = []
    self.labelsx = []
  #file_pathはメタCSVでーたのこと
  
  def flow_from_directory(self, file_path, classes, batch_size=4):
    blunc=np.empty((batch_size, 2048, 2048, 3), np.float32)
    blunc_l=np.empty((batch_size), np.float32)
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes = {v: i for i, v in enumerate(sorted(classes))}
    file_path = pd.read_csv(file_path)
    boss_dir = "/mnt/Doc"
    while True:
      # ディレクトリから画像のパスを取り出す
      count = 0
      bin8 = np.arange(batch_size,length,batch_size)
      hoge = np.arange(length)
      #np.random.shuffle(hoge)
      batch10 = np.split(hoge, bin8)
      #print(len(batch10))
      for ic, bbb in enumerate(batch10):
      for i in range(len(file_path)):
        # 画像を読み込みRGBへの変換、Numpyへの変換を行い、配列(self.iamges)に格納
        path = file_path.loc[i]["path1"]
        print(i)
        with Image.open(path) as f:
          self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
          # ファイル名からラベルを取り出し、配列(self.labels)に格納
          #_, y = path.stem.split('_')
        label = file_path.loc[i]["type"]
        self.labels.append(to_categorical(classes[label],len(classes)))
        self.labelsx.append(to_categorical(classes[label],len(classes)))
          # ここまでを繰り返し行い、batch_sizeの数だけ配列(self.iamges, self.labels)に格納
          # batch_sizeの数だけ格納されたら、戻り値として返し、配列(self.iamges, self.labels)を空にする
        if len(self.images)== batch_size:
          #print(len(self.images))
          inputs = np.asarray(self.images,dtype=np.float32) 
          targets = np.asarray(self.labels,dtype=np.float32)
            #print(len(self.images))
          count+=1
            #if count%1000==0:
            #print("n*1000!")
          self.reset()
          yield inputs, targets

In [0]:
class ImageDataGenerator1(object):
  
  def __init__(self):
    self.reset()
  def reset(self):
    self.images = []
    self.labels = []
  def flow_from_directory(self, file_path, classes, batch_size=8):
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes = {v: i for i, v in enumerate(sorted(classes))}
    file_path1 = pd.read_csv(file_path)
    length = len(file_path1)
    while True:
      # ディレクトリから画像のパスを取り出すcount = 0
      count = 0
      bin8 = np.arange(batch_size,length,batch_size)
      hoge = np.arange(length)
      #np.random.shuffle(hoge)
      batch10 = np.split(hoge, bin8)
      #print(len(batch10))
      for ic, bbb in enumerate(batch10):
        print(bbb)
        #print(str(ic),"回目です")
        count = 0
        for ib,bb in enumerate(bbb):
          path = file_path1.loc[ib]["path1"]
          with Image.open(path) as f:
            self.images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
          # ファイル名からラベルを取り出し、配列(self.labels)に格納
          #_, y = path.stem.split('_')
          label = file_path1.loc[ib]["type"]
          self.labels.append(to_categorical(classes[label],len(classes)))
          #self.labelsx.append(to_categorical(classes[label],len(classes)))
            #print(ib)
          if len(self.labels)== batch_size:
            #print(len(self.images))
            inputs = np.asarray(self.images,dtype=np.float32) 
            targets = np.asarray(self.labels,dtype=np.float32)
            print(targets)
            #print(len(self.images))
            #if count%1000==0:
            #print("n*1000!")
            self.reset()
            yield inputs, targets
      
                    
              

In [0]:
class ImageDataGenerator2(object):
    def __init__(self):
        self.images = []
        self.labels = []
        #file_pathはメタCSVでーたのこと

    def flow_from_directory(self, file_path, classes, batch_size=16):
        tmp_images = np.zeros((batch_size, 2048, 2048, 3))
        tmp_labels = np.zeros((batch_size, 3))

        # LabelEncode(classをint型に変換)するためのdict\n",
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path1 = pd.read_csv(file_path)
        length = len(file_path1)
        while True:
            # ディレクトリから画像のパスを取り出すcount = 0
            count = 0
            bin8 = np.arange(batch_size,length,batch_size)
            hoge = np.arange(length)
            np.random.shuffle(hoge)
            batch10 = np.split(hoge, bin8)
            #print(len(batch10))
            for ic, bbb in enumerate(batch10):
                #print(bbb)
                #print(str(ic),"回目です")
                for ib,bb in enumerate(bbb):
                    #print(ib, bb)
                    #print(bb)
                    path = file_path1.loc[bb]["path1"]
                    with Image.open(path) as f:
                        tmp_images[ib] = np.asarray(f, dtype=np.float32)/255
                    label = file_path1.loc[bb]["type"]
                    tmp_labels[ib] = to_categorical(classes[label],len(classes))
                #print(tmp_labels, np.mean(tmp_images, axis=(1,2,3)))
                yield tmp_images, tmp_labels
      #validation_data=test_datagen.flow_from_directory(test_dir, classes),
    #validation_steps=104/batchsize,
    #callbacks = [modelCheckpoint]             

In [0]:
class ImageDataGenerator1(object):
    def __init__(self):
        self.images = []
        self.labels = []
        #file_pathはメタCSVでーたのこと

    def flow_from_directory(self, file_path, classes, batch_size=8):
        tmp_images = np.zeros((batch_size, 2048, 2048, 3))
        tmp_labels = np.zeros((batch_size, 3))

        # LabelEncode(classをint型に変換)するためのdict\n",
        classes = {v: i for i, v in enumerate(sorted(classes))}
        file_path1 = pd.read_csv(file_path)
        length = len(file_path1)
        while True:
            # ディレクトリから画像のパスを取り出すcount = 0
            count = 0
            bin8 = np.arange(batch_size,length,batch_size)
            hoge = np.arange(length)
            np.random.shuffle(hoge)
            batch10 = np.split(hoge, bin8)
            #print(len(batch10))
            for ic, bbb in enumerate(batch10):
                #print(bbb)
                #print(str(ic),"回目です")
                for ib,bb in enumerate(bbb):
                    #print(ib, bb)
                    #print(bb)
                    path = file_path1.loc[bb]["path1"]
                    with Image.open(path) as f:
                        tmp_images[ib] = np.asarray(f, dtype=np.float32)/255
                    label = file_path1.loc[bb]["type"]
                    tmp_labels[ib] = to_categorical(classes[label],len(classes))
                #print(tmp_labels, np.mean(tmp_images, axis=(1,2,3)))
                yield tmp_images, tmp_labels
      #validation_data=test_datagen.flow_from_directory(test_dir, classes),
    #validation_steps=104/batchsize,
    #callbacks = [modelCheckpoint]             

In [0]:
#test_dir = pathlib.Path('/path/to/test/')
#x = pd.read_csv("/mnt/Doc/2048pix_tissue_all/RGBmean<190.csv")
train_dir = "/content/drive/My Drive/train_7136.csv"
test_dir = "/content/drive/My Drive/val_104.csv"
train_len = len(pd.read_csv(train_dir))
#print(train_len)
test_len = len(pd.read_csv(test_dir))
test_datagen = ImageDataGenerator1()
train_datagen = ImageDataGenerator2()
classes =["PI","PP","TRU"]
batchsize_train = 16
batchsize_val = 8
model.fit_generator(
    generator=train_datagen.flow_from_directory(train_dir, classes),
    steps_per_epoch=7136/batchsize_train,
    epochs=1000,
    verbose=1,
    callbacks = [modelCheckpoint] ,
    validation_data=test_datagen.flow_from_directory(test_dir, classes),
    validation_steps=104/batchsize_val
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
147/446 [========>.....................] - ETA: 35:52 - loss: 1.3687 - acc: 0.3520

In [0]:
validation_data=test_datagen.flow_from_directory(test_dir, classes),
    validation_steps=104/batchsize,
    callbacks = [modelCheckpoint] 

In [0]:
model.load_weights("/content/drive/My Drive/research_/weight/weight_0529_02.h5")

In [0]:
test_dir = "/content/drive/My Drive/val_104.csv"
test_datagen = ImageDataGenerator1()
classes =["PI","PP","TRU"]
batchsize = 8
predict = model.evaluate_generator(test_datagen.flow_from_directory(test_dir,classes),
                                   steps=104/batchsize,
                                   verbose=1
                                  )

[66 92 78 60 44  7 95 31]
[57 51 70 61 99 84 96 77]
13/13 [==============================] - 13s 992ms/step


In [0]:
predict

[3.6657338569057174e-06, 1.0]

In [0]:
test_dir = "/content/drive/My Drive/val_104.csv"
test_datagen = ImageDataGenerator1()
classes =["PI","PP","TRU"]
batchsize = 8
predict = model.predict_generator(test_datagen.flow_from_directory(test_dir,classes),
                                   steps=104/batchsize,
                                   verbose=1
                                  )

[0 1 2 3 4 5 6 7]
[ 8  9 10 11 12 13 14 15]
13/13 [==============================] - 13s 1s/step


In [0]:
predict[3].argmax()

0

In [0]:
x = []
classes =["PI","PP","TRU"]
classes = {v: i for i, v in enumerate(sorted(classes))}
for i in range(len(predict)):
  if predict[i].argmax() == 0:
    x.append("PI")
  if predict[i].argmax() == 1:
    x.append("PP")
  if predict[i].argmax() == 2:
    x.append("TRU")
  

In [0]:
classes =["PI","PP","TRU"]
classes = {v: i for i, v in enumerate(sorted(classes))}


In [0]:
classes

{'PI': 0, 'PP': 1, 'TRU': 2}

In [0]:
classes

{0: 'PI', 1: 'PP', 2: 'TRU'}

In [0]:
y = pd.read_csv("/content/drive/My Drive/val_104.csv",sep=',')
print(y["type"].head(20))
print(y.loc[21])

0      PI
1      PP
2      PP
3      PI
4      PP
5      PP
6      PI
7     TRU
8      PP
9      PI
10     PP
11     PI
12     PP
13     PI
14     PI
15    TRU
16     PP
17     PI
18     PI
19     PI
Name: type, dtype: object
Unnamed: 0                                                   21
type                                                         PI
ID                                      TCGA-97-7554-01A-01-BS1
file_pass     /home/kanayalab/Documents/2048pix tissue all/P...
Split                                                      test
high                                                       2048
wide                                                       2048
depth                                                         3
sd                                                      49.6476
RGBmean                                                 181.468
path1         /content/drive/My Drive/2048pix_tissue_choice/...
Name: 21, dtype: object


In [0]:
x[21]

'PP'